In [1]:
import requests,pandas,geopandas,shapely,os,re

In [2]:
def converter(results_dir,label,url,relevant_fields):
    
    elf.write('Processing {} ...\n'.format(label))
    
    # 1. read url
    response=requests.get(url)
    data=response.json()
    
    # 2. create dictionary
    dictionary={}
    for field in relevant_fields:
        dictionary[field]=[]
        
    message='\t Detected {} locations.\n'.format(len(data['items']))
    elf.write(message)

    for item in data['items']:    

        item_id=item['id']
        titol=item['titol']
        
        # working with phones
        clean_phones=[]
        for element in item['telefons']:
            if (element != '') and (element[:3] != 'Ext'):
                
                element=element.replace('T1','')
                if element[:2] == '34':
                    element=element[3:]
                    
                element=re.sub('[^0-9]','',element)
                
                if (element[0] == '9') or (element[0] == '6') or (element == '012') or (element == '112') or (element == '092') or (element[0] == '7') or (element[0] == '8'):
                    clean_phones.append(element)
                else:
                    elf.write('WARNING: Unrecognized phone number for {}. Phone number not added to shapefile: {}\n'.format(titol,element))
        phones=', '.join(clean_phones)
        # end of working with phones

        try:
            utmx=item['utm']['utmx']
            utmy=item['utm']['utmy']

            dictionary['id'].append(item_id)
            dictionary['titol'].append(titol)
            dictionary['utx'].append(utmx)
            dictionary['uty'].append(utmy)
            dictionary['phone'].append(phones)

        except:
            message='WARNING: dropping {} because utm not found.\n'.format(titol)
            elf.write(message)
    
    # 3. create pandas dataframe and write csv
    df=pandas.DataFrame.from_dict(dictionary)
    df.to_csv('{}{}.txt'.format(results_dir,label),sep='\t')
    
    # 4. create geopandas dataframe
    geometry=[shapely.geometry.Point(xy) for xy in zip(df.utx,df.uty)]
    crs={'init':'epsg:25831'}
    gdf=geopandas.GeoDataFrame(df,crs=crs,geometry=geometry)
    
    # 5. create shapefile
    gdf.to_file('{}{}.shp'.format(results_dir,label))
    
    return None

# 0. User defined variables

In [3]:
url_file='/Volumes/GoogleDrive/My Drive/help/toni/data/url_serveisoberts.txt'
results_dir='/Volumes/GoogleDrive/My Drive/help/toni/results/all/'
error_log_file='/Volumes/GoogleDrive/My Drive/help/toni/results/error_log.txt'

In [4]:
relevant_fields=['id','titol','utx','uty','phone']

In [5]:
tags={
    'Ajuntaments':'ajuntaments',
    'Confraries':'conf_pesc',
    'atenci%C3%B3%20ciutadana':'of_aten_ciut',
    'agr%C3%A0ria':'eca',
    'capacitaci%C3%B3%20nauticopesquera':'ecanautica',
    'Hotels':'hotels_entitats',
    'Biblioteques':'biblioteca',
    'Cultura&adv':'cultura',
    'Educaci%C3%B3':'educacio',
    'Depend':'cme',
    'Bombers':'bombers',
    'Policia%20de%20Tr%C3%A0nsit%':'transit_mossos',
    'Policies%20Locals':'polic_local',
    'Centres%20sanitaris':'centres_sanitaris',
    'deixalleries':'deixalleries',
}

# 1. Open error log file

In [6]:
elf=open(error_log_file,'w')

# 2. Read data

In [7]:
# not working because:
# DriverError: '/vsimem/b90a75d3a81f4494ac7992a7be08428d' not recognized as a supported file format.

# geopandas.read_file(url)

In [8]:
urls=[]
with open(url_file,'r') as f:
    for line in f:
        v=line.split()
        urls.append(v[0])
print('URLs detected: {}'.format(len(urls)))

URLs detected: 15


In [9]:
# check that each tag is uniquely URL
for tag in tags:
    found=sum(tag in s for s in urls)
    if found != 1:
        raise ValueError('Tag found more than in one URL.')

# 3. Iterate URLs

In [10]:
for url in urls:
    
    # select the label for the output file
    for tag in tags:
        if tag in url:
            label=tags[tag]
            
    # convert json to shapefile
    converter(results_dir,label,url,relevant_fields)

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/GoogleDrive/My Drive/help/toni/results/all/ajuntaments.txt'

# 4. Close error log file

In [ ]:
elf.close()